# 새로운 베이스라인

이 베이스라인은 다음과 같은 문제를 해결합니다.
1. train 데이터 안에서 중복값을 제거합니다.
2. 결측치를 해결합니다.
  1. 10분내버스정류장수와 지하철수에 대해 확실한 값을 처리 (4개 항목, 1개의 단지코드)
  2. 자격유형의 결측치를 처리합니다.
  3. 임대료의 결측치를 그대로 두거나 이상치(음수)로 처리합니다.
  4. 나머지 결측치에 대해 fillna를 실시합니다.
3. groupby를 통해 모든 columns이 같지만 전용면적별세대수만 다른 경우에 대해 합을 구합니다.
4. 단지코드가 같은 여러 행의 예측값을 단지별 세대수의 합으로 곱한 후 총 세대수로 나눠 제출합니다.
5. 주상복합의 경우 상가데이터를 제거하기
  - 주상복합의 경우 아파트와 상가의 등록차량수가 일치하기 때문에 제거해주고 학습시킵니다.
  - 주상복합은 총세대수가 많기 때문에 상가를 제거하는 것이 큰 문제가 없을 것이라고 생각합니다. (이 것에 대해 데이터를 직접 분석해본 것은 아님)

---

### 추가로 이러한 문제를 해결할 수 있습니다. (우선순위 낮음)
1. 단지코드 단순화
- 단지코드당 행의 개수를 1개로 한정합니다.
- 제거하는 경우, 전용면적, 전용면적별 세대수 등을 제거하고 새로운 피처로 단지코드 내 한 세대의 평균 전용면적을 구해줍니다.
  - 세대 당 평균 전용면적: sum([전용면적별 전용면적 * 세대수 여러개]) / 총 세대수
- 문제점: 임대료 등은 천차만별인데 이를 통일성있게 반영할 방법이 없다. 임대료나 임대보증금이 모두 있는 경우 상관없지만 (그 아파트의 세대당/평당 임대료를 구할 수 있음) 만약 장기전세이거나 결측치가 있는 경우 이를 반영하기 어렵다.
  

In [1]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [2]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [3]:
import pandas as pd
import numpy as np

In [4]:
# ### 한글 폰트 설정
# import matplotlib
# from matplotlib import font_manager, rc
# import matplotlib.pyplot as plt
# import platform
# path = "C:/Windows/Fonts/malgun.ttf"
# if platform.system() == "Windows":
#     font_name = font_manager.FontProperties(fname=path).get_name()
#     rc('font', family=font_name)
# elif platform.system()=="Darwin":
#     rc('font', family='AppleGothic')
# elif platform.system()=="Linux":
#     rc('font', family='NanumBarunGothic')
# else:
#     print("Unknown System")
# matplotlib.rcParams['axes.unicode_minus'] = False

In [5]:
# PATH = "/content/gdrive/MyDrive/Colab Notebooks/parking_demand"
# train = pd.read_csv(PATH + "/train_df_errno.csv")
# test = pd.read_csv(PATH + "/test_df.csv")
# sub = pd.read_csv(PATH + "/sample_submission.csv")
# age = pd.read_csv(PATH + "/age_gender_info.csv")

In [6]:
train=pd.read_csv('./train_df_errno.csv')
test=pd.read_csv('./test_df.csv')
sub=pd.read_csv('./sample_submission.csv')
age=pd.read_csv('./age_gender_info.csv')
train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [7]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [8]:
train.shape

(2896, 15)

### train 데이터 내에서 모든 중복값 확인하기

In [9]:
train[train.duplicated(keep=False)]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
12,C1407,1216,아파트,대전광역시,국민임대,41.58,70,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0
13,C1407,1216,아파트,대전광역시,국민임대,41.58,70,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0
14,C1407,1216,아파트,대전광역시,국민임대,46.36,170,13.0,A,36048000,198250,1.0,1.0,1285.0,1064.0
15,C1407,1216,아파트,대전광역시,국민임대,46.36,170,13.0,A,36048000,198250,1.0,1.0,1285.0,1064.0
16,C1407,1216,아파트,대전광역시,국민임대,51.24,62,13.0,A,42056000,253520,1.0,1.0,1285.0,1064.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,C1803,100,아파트,울산광역시,행복주택,36.16,9,13.0,J,46400000,220780,0.0,2.0,57.0,78.0
2750,C2362,1509,아파트,경기도,행복주택,21.96,144,46.0,J,19760000,83000,0.0,2.0,1055.0,196.0
2751,C2362,1509,아파트,경기도,행복주택,21.96,144,46.0,J,19760000,83000,0.0,2.0,1055.0,196.0
2755,C2362,1509,아파트,경기도,행복주택,26.85,48,46.0,J,23520000,99000,0.0,2.0,1055.0,196.0


중복값을 제거합니다 - 개수가 2896에서 2576으로 줄어듦

In [10]:
train.drop_duplicates(inplace=True)
train.shape

(2576, 15)

In [11]:
test.drop_duplicates(inplace=True)
test.shape

(936, 14)

In [12]:
train_test = pd.concat([train, test], join="inner", ignore_index=True)
train_test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
3508,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
3509,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
3510,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [13]:
train_test[train_test["10분내버스정류장수"].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
1985,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0
1986,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0
1987,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0
1988,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0


In [14]:
train_test[train_test["단지코드"] == "N2431"]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
1985,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0
1986,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0
1987,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0
1988,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0
2042,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0
2043,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0


In [15]:
train_test.loc[train_test["단지코드"] == "N2431", "10분내버스정류장수"] = 2

In [16]:
train_test.loc[train_test["단지코드"] == "N2431", "10분내지하철수"] = 0
train_test.loc[train_test["단지코드"] == "N2431"]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
1985,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,0.0,2.0,1066.0
1986,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,0.0,2.0,1066.0
1987,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,0.0,2.0,1066.0
1988,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,0.0,2.0,1066.0
2042,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0
2043,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0


In [17]:
train_test.loc[train_test['10분내지하철수'].isnull(), "10분내지하철수"] = -9
train_test["10분내지하철수"].value_counts()

 0.0    2892
 1.0     372
-9.0     182
 2.0      60
 3.0       6
Name: 10분내지하철수, dtype: int64

In [18]:
train_test.loc[train_test['10분내지하철수'] == 3]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
1746,C1987,291,아파트,부산광역시,국민임대,46.53,31,26.0,A,44387000,368980,3.0,4.0,257.0
1747,C1987,291,아파트,부산광역시,국민임대,46.53,128,26.0,A,44387000,368980,3.0,4.0,257.0
1748,C1987,291,아파트,부산광역시,국민임대,46.53,35,26.0,A,44387000,368980,3.0,4.0,257.0
1749,C1987,291,아파트,부산광역시,국민임대,46.53,6,26.0,A,44387000,368980,3.0,4.0,257.0
1750,C1987,291,아파트,부산광역시,국민임대,51.55,19,26.0,A,50775000,422950,3.0,4.0,257.0
1751,C1987,291,아파트,부산광역시,국민임대,51.55,72,26.0,A,50775000,422950,3.0,4.0,257.0


In [19]:
train_test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         406
임대료           406
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [20]:
train_test["공급유형"].value_counts()

국민임대         2319
임대상가          397
행복주택          310
공공임대(10년)     214
영구임대          191
공공임대(50년)      44
공공임대(분납)       18
장기전세            9
공공분양            7
공공임대(5년)        3
Name: 공급유형, dtype: int64

##### 자격유형의 결측치를 처리할까?
1. 지금 자격유형 처리 -> 데이콘 점수가 올라갈 수 있다.
2. 지금 우리만의 논리로 자격유형 처리 (검색에 의존하지 않고) -> 그 논리는 항상 정확할까?

응 필요없어~ 캣부스트 쓸거야 -> 캣부스트 nan 안돼~

In [21]:
train_test.groupby(["공급유형", "자격유형"]).count()

단지코드  총세대수  임대건물구분    지역  전용면적  전용면적별세대수   공가수  임대보증금   임대료  \
공급유형      자격유형                                                                
공공분양      D        7     7       7     7     7         7     7      0     0   
공공임대(10년) A      214   214     214   214   214       214   214    214   214   
공공임대(50년) A       44    44      44    44    44        44    44     44    44   
공공임대(5년)  A        3     3       3     3     3         3     3      3     3   
공공임대(분납)  A       18    18      18    18    18        18    18     18    18   
국민임대      A     2000  2000    2000  2000  2000      2000  2000   2000  2000   
          B       18    18      18    18    18        18    18     18    18   
          E       44    44      44    44    44        44    44     44    44   
          G       10    10      10    10    10        10    10     10    10   
          H      246   246     246   246   246       246   246    246   246   
영구임대      A        2     2       2     2     2         2     2      2     2   
          C      124   124     124   124   124       124   124    124   124   
          D        2     2       2     2     2         2     2      0     0   
          E        3     3       3     3     3         3     3      3     3   
          F        3     3       3     3     3         3     3      3     3   
          I       56    56      56    56    56        56    56     56    56   
임대상가      D      397   397     397   397   397       397   397      0     0   
장기전세      A        9     9       9     9     9         9     9      9     9   
행복주택      J      172   172     172   172   172       172   172    172   172   
          K       49    49      49    49    49        49    49     49    49   
          L       45    45      45    45    45        45    45     45    45   
          M        4     4       4     4     4         4     4      4     4   
          N       39    39      39    39    39        39    39     39    39   
          O        1     1       1     1     1         1     1      1     1   

                10분내지하철수  10분내버스정류장수  단지내주차면수  
공급유형      자격유형                                 
공공분양      D            7           7        7  
공공임대(10년) A          214         214      214  
공공임대(50년) A           44          44       44  
공공임대(5년)  A            3           3        3  
공공임대(분납)  A           18          18       18  
국민임대      A         2000        2000     2000  
          B           18          18       18  
          E           44          44       44  
          G           10          10       10  
          H          246         246      246  
영구임대      A            2           2        2  
          C          124         124      124  
          D            2           2        2  
          E            3           3        3  
          F            3           3        3  
          I           56          56       56  
임대상가      D          397         397      397  
장기전세      A            9           9        9  
행복주택      J          172         172      172  
          K           49          49       49  
          L           45          45       45  
          M            4           4        4  
          N           39          39       39  
          O            1           1        1

In [22]:
train_test.loc[train_test["자격유형"].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
2740,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
2783,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [23]:
train_test.loc[2740, "자격유형"] = "A"
train_test.loc[2783, "자격유형"] = "C"
train_test.loc[train_test["자격유형"].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


### 임대료 결측치

In [24]:
train_test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         406
임대료           406
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [25]:
train_test["임대료"].value_counts().sort_index()

-         27
100000     5
100260     1
100270     1
100320     2
          ..
99000      6
99080      3
99100      1
99720      2
99850      3
Name: 임대료, Length: 1255, dtype: int64

In [26]:
train_test["임대보증금"].value_counts().sort_index()

-            18
10000000      2
100400000     1
10043000      1
10060000      2
             ..
99261000      2
9930000       1
9972000       5
9984000       3
9985000       3
Name: 임대보증금, Length: 1214, dtype: int64

In [27]:
train_test.loc[train_test["임대료"] == "-", "임대료"] = 0
train_test.loc[train_test["임대보증금"] == "-", "임대보증금"] = 0

In [29]:
train_test.loc[train_test["임대료"].isnull(), "임대료"] = -999
train_test.loc[train_test["임대보증금"].isnull(), "임대보증금"] = -999

In [30]:
train_test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금         0
임대료           0
10분내지하철수      0
10분내버스정류장수    0
단지내주차면수       0
dtype: int64

### 전용면적별 세대의 합계와 총세대수가 일치하지 않는 오류(by.강사님) 
 - 오류1 14세대 이하 처리
 - 오류1 94~452세대 10개 단지 처리
 - 나머지 null값처리

In [31]:
group1 = ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
          'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
          'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
          'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
          'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189' ]
for one1 in group1:
    train_test.loc[ train_test['단지코드'] == one1, "단지코드_Type" ] = 0

In [32]:
group2=['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804' ]
for one1 in group2:
     train_test.loc[ train_test['단지코드'] == one1, "단지코드_Type" ] = 1

In [33]:
train_test.loc[train_test['단지코드_Type'].isna(), "단지코드_Type"] = 2
print(train_test['단지코드_Type'].unique())

[2. 0. 1.]


### 전용면적에 제한두기: 5배수 전환, 100m면적 제한(by.DACON baseline)
 - 상가 제거로 상한과 하한의 이상치가 다 제거되었음 > 굳이 둘까 의문쓰
 - importance 비중이 본래 전용면적(1.14)보다 훨씬 떨어지는 것(0.54)을 보아 원본데이터로 사용하기로 결정!

In [34]:
# train_test['전용면적'] = train_test['전용면적']//5*5

In [35]:
# #상한제한
# idx = train_test[train_test['전용면적']>100].index
# train_test.loc[idx, '전용면적'] = 100

In [36]:
# #하한제한
# idx = train_test[train_test['전용면적']<15].index
# train_test.loc[idx, '전용면적'] = 15

In [ ]:
# train_test['전용면적'].unique()

##### 소득분위

In [37]:
train_test.loc[(train_test["자격유형"]=='C') | (train_test["자격유형"]=='E') | (train_test["자격유형"]=='F') | (train_test["자격유형"]=='I') , "소득분위"] =1
train_test.loc[(train_test["자격유형"]=='B') | (train_test["자격유형"]=='G') | (train_test["자격유형"]=='H'), "소득분위"] =2
train_test.loc[(train_test["자격유형"]=='A'), "소득분위"] =3
train_test.loc[(train_test["자격유형"]=='J') | (train_test["자격유형"]=='K') | (train_test["자격유형"]=='L') |(train_test["자격유형"]=='M') |(train_test["자격유형"]=='N')|(train_test["자격유형"]=='O'), "소득분위"] =4
train_test.loc[(train_test["자격유형"]=='D'), "소득분위"] =5

In [38]:
train_test['소득분위'].unique()

array([3., 2., 1., 5., 4.])

### 주차면수 대비 총세대수, 거주율

In [39]:
#주차면수 대비 총세대수가 높을 수록 주차수요가 높을것이다
train_test['주차면수 대비 총세대수']=train_test['단지내주차면수']/train_test['총세대수']

In [40]:
#거주율이 높을 수록 주차수요가 높을 것이다.
train_test['거주율']=1-(train_test['공가수']/train_test['총세대수'])

### 단지분류 : 소,중,대단지
### 단지분류lbl : 1, 2, 3

In [41]:
train_test.loc[train_test['총세대수']<600,'단지분류'] = '소단지'
train_test.loc[(600 <= train_test["총세대수"]) & (train_test["총세대수"] <= 1199),'단지분류']='중단지'
train_test.loc[(1200 <= train_test["총세대수"]),'단지분류']='대단지'

In [42]:
mapping = {'소단지' : 1,'중단지' : 2, '대단지' : 3}
train_test['단지분류lbl'] = train_test['단지분류'].map(mapping)

##### 타입바꿔주기

In [43]:
train_test.dtypes

단지코드             object
총세대수              int64
임대건물구분           object
지역               object
공급유형             object
전용면적            float64
전용면적별세대수          int64
공가수             float64
자격유형             object
임대보증금            object
임대료              object
10분내지하철수        float64
10분내버스정류장수      float64
단지내주차면수         float64
단지코드_Type       float64
소득분위            float64
주차면수 대비 총세대수    float64
거주율             float64
단지분류             object
단지분류lbl           int64
dtype: object

In [66]:
train_test['단지분류lbl']

0       1
1       1
2       1
3       1
4       1
       ..
3507    2
3508    1
3509    1
3510    1
3511    1
Name: 단지분류lbl, Length: 3512, dtype: int64

In [45]:
train_test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수',
       '단지코드_Type', '소득분위', '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl'],
      dtype='object')

In [44]:
print(np.where(train_test.dtypes != np.float)[0])

[ 0  1  2  3  4  6  8  9 10 18 19]


In [69]:
# train_test[["총세대수", "전용면적별세대수", "공가수", "임대보증금", "임대료", "10분내지하철수", "10분내버스정류장수", "단지내주차면수"]] = \
# train_test[["총세대수", "전용면적별세대수", "공가수", "임대보증금", "임대료", "10분내지하철수", "10분내버스정류장수", "단지내주차면수"]].astype(int)

# nan값 때문에 float로 남겨둬야 한다.: 10분내지하철,임대보증금,임대료?
# 전용면적,단지코드_Type은 int로

train_test[['전용면적','10분내지하철수','주차면수 대비 총세대수','거주율','임대보증금','임대료']]= \
train_test[['전용면적','10분내지하철수','주차면수 대비 총세대수','거주율','임대보증금','임대료']].astype(float)

train_test[["총세대수", "전용면적별세대수", "공가수","10분내버스정류장수", "단지내주차면수",'단지코드_Type','소득분위','단지분류lbl']] = \
train_test[["총세대수", "전용면적별세대수", "공가수","10분내버스정류장수", "단지내주차면수",'단지코드_Type','소득분위','단지분류lbl']].astype(int)

In [92]:
train_test.dtypes

단지코드             object
총세대수              int64
임대건물구분           object
지역               object
공급유형             object
전용면적            float64
공가수               int64
자격유형             object
임대보증금           float64
임대료             float64
10분내지하철수        float64
10분내버스정류장수        int64
단지내주차면수           int64
단지코드_Type         int64
소득분위              int64
주차면수 대비 총세대수    float64
거주율             float64
단지분류             object
단지분류lbl           int64
전용면적별세대수          int32
단지별전용면적평균       float64
dtype: object

### 상가 제거하기

상가를 제거하는 것만으로도 전용면적의 이상치를 제거할 수 있다. (최대 최소)

In [71]:
train_test.shape

(3512, 20)

In [72]:
train_test = train_test.drop(train_test[train_test["임대건물구분"] == "상가"].index)

### groupby를 통해서

In [73]:
train_test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수',
       '단지코드_Type', '소득분위', '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl'],
      dtype='object')

In [74]:
train_test = train_test.groupby(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수',
       '단지코드_Type', '소득분위','주차면수 대비 총세대수','거주율','단지분류','단지분류lbl'], sort=False, as_index=False)["전용면적별세대수"].sum()

In [75]:
train_test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,단지코드_Type,소득분위,주차면수 대비 총세대수,거주율,단지분류,단지분류lbl,전용면적별세대수
0,C2515,545,아파트,경상남도,국민임대,33.48,17,A,9216000.0,82940.0,0.0,3,624,2,3,1.144954,0.968807,소단지,1,276
1,C2515,545,아파트,경상남도,국민임대,39.60,17,A,12672000.0,107130.0,0.0,3,624,2,3,1.144954,0.968807,소단지,1,80
2,C2515,545,아파트,경상남도,국민임대,46.90,17,A,18433000.0,149760.0,0.0,3,624,2,3,1.144954,0.968807,소단지,1,57
3,C2515,545,아파트,경상남도,국민임대,51.97,17,A,23042000.0,190090.0,0.0,3,624,2,3,1.144954,0.968807,소단지,1,132
4,C1407,1216,아파트,대전광역시,국민임대,30.95,13,A,15620000.0,127350.0,1.0,1,1285,2,3,1.056743,0.989309,대단지,3,288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,C1267,675,아파트,경상남도,행복주택,26.85,38,L,0.0,0.0,0.0,1,467,0,4,0.691852,0.943704,중단지,2,74
2193,C1267,675,아파트,경상남도,행복주택,36.77,38,L,0.0,0.0,0.0,1,467,0,4,0.691852,0.943704,중단지,2,126
2194,C2189,382,아파트,전라북도,국민임대,29.19,45,H,6872000.0,106400.0,0.0,2,300,0,2,0.785340,0.882199,소단지,1,116
2195,C2189,382,아파트,전라북도,국민임대,39.45,45,H,13410000.0,144600.0,0.0,2,300,0,2,0.785340,0.882199,소단지,1,202


### 새로운 피처 만들기 - 단지별전용면적평균 구해주기

In [76]:
train_test_temp = train_test.copy()
train_test_temp["전용면적*전용면적별세대수"] = train_test_temp['전용면적'] * train_test_temp['전용면적별세대수'] / train_test_temp["총세대수"]
train_test["단지별전용면적평균"] =\
train_test_temp.groupby("단지코드")["전용면적*전용면적별세대수"].transform(np.sum)
train_test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,10분내버스정류장수,단지내주차면수,단지코드_Type,소득분위,주차면수 대비 총세대수,거주율,단지분류,단지분류lbl,전용면적별세대수,단지별전용면적평균
0,C2515,545,아파트,경상남도,국민임대,33.48,17,A,9216000.0,82940.0,...,3,624,2,3,1.144954,0.968807,소단지,1,276,40.260220
1,C2515,545,아파트,경상남도,국민임대,39.60,17,A,12672000.0,107130.0,...,3,624,2,3,1.144954,0.968807,소단지,1,80,40.260220
2,C2515,545,아파트,경상남도,국민임대,46.90,17,A,18433000.0,149760.0,...,3,624,2,3,1.144954,0.968807,소단지,1,57,40.260220
3,C2515,545,아파트,경상남도,국민임대,51.97,17,A,23042000.0,190090.0,...,3,624,2,3,1.144954,0.968807,소단지,1,132,40.260220
4,C1407,1216,아파트,대전광역시,국민임대,30.95,13,A,15620000.0,127350.0,...,1,1285,2,3,1.056743,0.989309,대단지,3,288,28.939474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,C1267,675,아파트,경상남도,행복주택,26.85,38,L,0.0,0.0,...,1,467,0,4,0.691852,0.943704,중단지,2,74,32.054015
2193,C1267,675,아파트,경상남도,행복주택,36.77,38,L,0.0,0.0,...,1,467,0,4,0.691852,0.943704,중단지,2,126,32.054015
2194,C2189,382,아파트,전라북도,국민임대,29.19,45,H,6872000.0,106400.0,...,2,300,0,2,0.785340,0.882199,소단지,1,116,36.986230
2195,C2189,382,아파트,전라북도,국민임대,39.45,45,H,13410000.0,144600.0,...,2,300,0,2,0.785340,0.882199,소단지,1,202,36.986230


## train, test 두 개로 다시 나눠주기

In [77]:
test.loc[0, "단지코드"]

'C1072'

In [78]:
# 테스트 데이터의 첫부분
train_test[train_test["단지코드"] == "C1072"]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,10분내버스정류장수,단지내주차면수,단지코드_Type,소득분위,주차면수 대비 총세대수,거주율,단지분류,단지분류lbl,전용면적별세대수,단지별전용면적평균
1605,C1072,754,아파트,경기도,국민임대,39.79,14,H,22830000.0,189840.0,...,2,683,2,2,0.905836,0.981432,중단지,2,116,48.269164
1606,C1072,754,아파트,경기도,국민임대,46.81,14,A,36048000.0,249930.0,...,2,683,2,3,0.905836,0.981432,중단지,2,30,48.269164
1607,C1072,754,아파트,경기도,국민임대,46.90,14,H,36048000.0,249930.0,...,2,683,2,2,0.905836,0.981432,중단지,2,232,48.269164
1608,C1072,754,아파트,경기도,국민임대,51.46,14,H,43497000.0,296780.0,...,2,683,2,2,0.905836,0.981432,중단지,2,60,48.269164
1609,C1072,754,아파트,경기도,국민임대,51.71,14,H,43497000.0,296780.0,...,2,683,2,2,0.905836,0.981432,중단지,2,51,48.269164
1610,C1072,754,아파트,경기도,국민임대,51.96,14,H,43497000.0,296780.0,...,2,683,2,2,0.905836,0.981432,중단지,2,265,48.269164


In [79]:
train_test_index = train_test[train_test["단지코드"] == "C1072"].index[0]
train_df = train_test.iloc[0:train_test_index]
test = train_test.iloc[train_test_index:]
train_test.shape, train_df.shape, test.shape

((2197, 21), (1605, 21), (592, 21))

In [80]:
num_of_cars = train.set_index("단지코드")["등록차량수"]
num_of_cars

단지코드
C2515    205.0
C2515    205.0
C2515    205.0
C2515    205.0
C2515    205.0
         ...  
C2532    146.0
C2532    146.0
C2532    146.0
C2532    146.0
C2532    146.0
Name: 등록차량수, Length: 2576, dtype: float64

In [81]:
train = train_df.join(num_of_cars, on="단지코드").drop_duplicates()

In [82]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
공가수             0
자격유형            0
임대보증금           0
임대료             0
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
단지코드_Type       0
소득분위            0
주차면수 대비 총세대수    0
거주율             0
단지분류            0
단지분류lbl         0
전용면적별세대수        0
단지별전용면적평균       0
등록차량수           0
dtype: int64

In [83]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
공가수             0
자격유형            0
임대보증금           0
임대료             0
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
단지코드_Type       0
소득분위            0
주차면수 대비 총세대수    0
거주율             0
단지분류            0
단지분류lbl         0
전용면적별세대수        0
단지별전용면적평균       0
dtype: int64

In [84]:
train.shape

(1605, 22)

In [85]:
test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '단지코드_Type', '소득분위',
       '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl', '전용면적별세대수', '단지별전용면적평균'],
      dtype='object')

In [86]:
train.dtypes

단지코드             object
총세대수              int64
임대건물구분           object
지역               object
공급유형             object
전용면적            float64
공가수               int64
자격유형             object
임대보증금           float64
임대료             float64
10분내지하철수        float64
10분내버스정류장수        int64
단지내주차면수           int64
단지코드_Type         int64
소득분위              int64
주차면수 대비 총세대수    float64
거주율             float64
단지분류             object
단지분류lbl           int64
전용면적별세대수          int32
단지별전용면적평균       float64
등록차량수           float64
dtype: object

## CatBoost 모델

In [87]:
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [88]:
train['log_등록차량수'] = np.log1p(train['등록차량수'])

In [89]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score

In [90]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '단지코드_Type', '소득분위',
       '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl', '전용면적별세대수', '단지별전용면적평균',
       '등록차량수', 'log_등록차량수'],
      dtype='object')

In [91]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1605 entries, 0 to 1604
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   단지코드          1605 non-null   object 
 1   총세대수          1605 non-null   int64  
 2   임대건물구분        1605 non-null   object 
 3   지역            1605 non-null   object 
 4   공급유형          1605 non-null   object 
 5   전용면적          1605 non-null   float64
 6   공가수           1605 non-null   int64  
 7   자격유형          1605 non-null   object 
 8   임대보증금         1605 non-null   float64
 9   임대료           1605 non-null   float64
 10  10분내지하철수      1605 non-null   float64
 11  10분내버스정류장수    1605 non-null   int64  
 12  단지내주차면수       1605 non-null   int64  
 13  단지코드_Type     1605 non-null   int64  
 14  소득분위          1605 non-null   int64  
 15  주차면수 대비 총세대수  1605 non-null   float64
 16  거주율           1605 non-null   float64
 17  단지분류          1605 non-null   object 
 18  단지분류lbl       1605 non-null 

In [93]:
cat_model = CatBoostRegressor(
    cat_features=['단지코드', '임대건물구분', '지역', '공급유형', '자격유형','단지분류'],
#     # loss_function='MAE',
    n_estimators=10000, 
#     # learning_rate=0.05, 
#     # random_state=42,
#     # task_type="GPU",
#     # depth=8,
#     nan_mode="Forbidden"
#     # od_type="IncToDec"
)

In [94]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '단지코드_Type', '소득분위',
       '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl', '전용면적별세대수', '단지별전용면적평균',
       '등록차량수', 'log_등록차량수'],
      dtype='object')

In [95]:
train.iloc[:, -1]

0       5.327876
1       5.327876
2       5.327876
3       5.327876
4       6.970730
          ...   
1600    4.990433
1601    4.990433
1602    4.990433
1603    4.990433
1604    4.990433
Name: log_등록차량수, Length: 1605, dtype: float64

In [96]:
train.iloc[:, :-2].columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '단지코드_Type', '소득분위',
       '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl', '전용면적별세대수', '단지별전용면적평균'],
      dtype='object')

In [97]:
train['전용면적별세대수'].shape

(1605,)

In [98]:
X = train.iloc[:, :-2]
y = train.iloc[:, -1]

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [100]:
cat_model.fit(
    X_train, y_train,
    eval_set = (X_test, y_test),
    logging_level="Verbose"
)

Learning rate set to 0.011367
0:	learn: 0.7711539	test: 0.7732401	best: 0.7732401 (0)	total: 166ms	remaining: 27m 36s
1:	learn: 0.7656325	test: 0.7676508	best: 0.7676508 (1)	total: 195ms	remaining: 16m 13s
2:	learn: 0.7603687	test: 0.7623840	best: 0.7623840 (2)	total: 227ms	remaining: 12m 35s
3:	learn: 0.7552538	test: 0.7575214	best: 0.7575214 (3)	total: 261ms	remaining: 10m 52s
4:	learn: 0.7499268	test: 0.7523158	best: 0.7523158 (4)	total: 288ms	remaining: 9m 35s
5:	learn: 0.7450221	test: 0.7477503	best: 0.7477503 (5)	total: 314ms	remaining: 8m 43s
6:	learn: 0.7395314	test: 0.7421666	best: 0.7421666 (6)	total: 342ms	remaining: 8m 8s
7:	learn: 0.7343979	test: 0.7369622	best: 0.7369622 (7)	total: 369ms	remaining: 7m 40s
8:	learn: 0.7292836	test: 0.7321212	best: 0.7321212 (8)	total: 396ms	remaining: 7m 19s
9:	learn: 0.7244386	test: 0.7276759	best: 0.7276759 (9)	total: 423ms	remaining: 7m 2s
10:	learn: 0.7192485	test: 0.7228000	best: 0.7228000 (10)	total: 453ms	remaining: 6m 51s
11:	learn

94:	learn: 0.4500818	test: 0.4593571	best: 0.4593571 (94)	total: 3.04s	remaining: 5m 17s
95:	learn: 0.4479440	test: 0.4571675	best: 0.4571675 (95)	total: 3.08s	remaining: 5m 17s
96:	learn: 0.4462624	test: 0.4553962	best: 0.4553962 (96)	total: 3.11s	remaining: 5m 17s
97:	learn: 0.4446517	test: 0.4537214	best: 0.4537214 (97)	total: 3.14s	remaining: 5m 17s
98:	learn: 0.4427957	test: 0.4520849	best: 0.4520849 (98)	total: 3.17s	remaining: 5m 16s
99:	learn: 0.4408475	test: 0.4499450	best: 0.4499450 (99)	total: 3.19s	remaining: 5m 16s
100:	learn: 0.4391475	test: 0.4481288	best: 0.4481288 (100)	total: 3.23s	remaining: 5m 16s
101:	learn: 0.4373947	test: 0.4463589	best: 0.4463589 (101)	total: 3.26s	remaining: 5m 15s
102:	learn: 0.4356903	test: 0.4446868	best: 0.4446868 (102)	total: 3.29s	remaining: 5m 15s
103:	learn: 0.4335900	test: 0.4426439	best: 0.4426439 (103)	total: 3.32s	remaining: 5m 16s
104:	learn: 0.4321009	test: 0.4411156	best: 0.4411156 (104)	total: 3.35s	remaining: 5m 15s
105:	learn:

187:	learn: 0.3349072	test: 0.3417859	best: 0.3417859 (187)	total: 5.92s	remaining: 5m 9s
188:	learn: 0.3341057	test: 0.3408743	best: 0.3408743 (188)	total: 5.95s	remaining: 5m 9s
189:	learn: 0.3332595	test: 0.3398742	best: 0.3398742 (189)	total: 5.98s	remaining: 5m 8s
190:	learn: 0.3325612	test: 0.3391660	best: 0.3391660 (190)	total: 6.05s	remaining: 5m 10s
191:	learn: 0.3319848	test: 0.3385342	best: 0.3385342 (191)	total: 6.09s	remaining: 5m 11s
192:	learn: 0.3314163	test: 0.3379606	best: 0.3379606 (192)	total: 6.12s	remaining: 5m 10s
193:	learn: 0.3305444	test: 0.3370811	best: 0.3370811 (193)	total: 6.15s	remaining: 5m 10s
194:	learn: 0.3298657	test: 0.3364299	best: 0.3364299 (194)	total: 6.17s	remaining: 5m 10s
195:	learn: 0.3293904	test: 0.3359252	best: 0.3359252 (195)	total: 6.19s	remaining: 5m 9s
196:	learn: 0.3286663	test: 0.3350553	best: 0.3350553 (196)	total: 6.22s	remaining: 5m 9s
197:	learn: 0.3279032	test: 0.3340735	best: 0.3340735 (197)	total: 6.25s	remaining: 5m 9s
198:	

281:	learn: 0.2834901	test: 0.2861685	best: 0.2861685 (281)	total: 8.7s	remaining: 4m 59s
282:	learn: 0.2831638	test: 0.2858502	best: 0.2858502 (282)	total: 8.73s	remaining: 4m 59s
283:	learn: 0.2828724	test: 0.2855081	best: 0.2855081 (283)	total: 8.76s	remaining: 4m 59s
284:	learn: 0.2825557	test: 0.2853417	best: 0.2853417 (284)	total: 8.79s	remaining: 4m 59s
285:	learn: 0.2822969	test: 0.2850832	best: 0.2850832 (285)	total: 8.81s	remaining: 4m 59s
286:	learn: 0.2819096	test: 0.2846023	best: 0.2846023 (286)	total: 8.84s	remaining: 4m 59s
287:	learn: 0.2816095	test: 0.2842842	best: 0.2842842 (287)	total: 8.87s	remaining: 4m 59s
288:	learn: 0.2812694	test: 0.2838873	best: 0.2838873 (288)	total: 8.9s	remaining: 4m 59s
289:	learn: 0.2810257	test: 0.2836107	best: 0.2836107 (289)	total: 8.93s	remaining: 4m 58s
290:	learn: 0.2806516	test: 0.2832060	best: 0.2832060 (290)	total: 8.96s	remaining: 4m 58s
291:	learn: 0.2803445	test: 0.2829225	best: 0.2829225 (291)	total: 8.98s	remaining: 4m 58s
2

374:	learn: 0.2576902	test: 0.2611112	best: 0.2611112 (374)	total: 11.5s	remaining: 4m 54s
375:	learn: 0.2574387	test: 0.2609079	best: 0.2609079 (375)	total: 11.5s	remaining: 4m 55s
376:	learn: 0.2572593	test: 0.2607884	best: 0.2607884 (376)	total: 11.6s	remaining: 4m 55s
377:	learn: 0.2570351	test: 0.2606566	best: 0.2606566 (377)	total: 11.6s	remaining: 4m 55s
378:	learn: 0.2567934	test: 0.2604336	best: 0.2604336 (378)	total: 11.6s	remaining: 4m 54s
379:	learn: 0.2565157	test: 0.2601567	best: 0.2601567 (379)	total: 11.6s	remaining: 4m 54s
380:	learn: 0.2562087	test: 0.2599957	best: 0.2599957 (380)	total: 11.7s	remaining: 4m 54s
381:	learn: 0.2561415	test: 0.2599419	best: 0.2599419 (381)	total: 11.7s	remaining: 4m 54s
382:	learn: 0.2558054	test: 0.2596938	best: 0.2596938 (382)	total: 11.7s	remaining: 4m 54s
383:	learn: 0.2556657	test: 0.2595677	best: 0.2595677 (383)	total: 11.7s	remaining: 4m 54s
384:	learn: 0.2555607	test: 0.2594566	best: 0.2594566 (384)	total: 11.8s	remaining: 4m 53s

...

9602:	learn: 0.0194682	test: 0.0921352	best: 0.0921352 (9602)	total: 4m 52s	remaining: 12.1s
9603:	learn: 0.0194675	test: 0.0921350	best: 0.0921350 (9603)	total: 4m 52s	remaining: 12s
9604:	learn: 0.0194672	test: 0.0921351	best: 0.0921350 (9603)	total: 4m 52s	remaining: 12s
9605:	learn: 0.0194642	test: 0.0921341	best: 0.0921341 (9605)	total: 4m 52s	remaining: 12s
9606:	learn: 0.0194582	test: 0.0921282	best: 0.0921282 (9606)	total: 4m 52s	remaining: 12s
9607:	learn: 0.0194545	test: 0.0921277	best: 0.0921277 (9607)	total: 4m 52s	remaining: 11.9s
9608:	learn: 0.0194532	test: 0.0921267	best: 0.0921267 (9608)	total: 4m 52s	remaining: 11.9s
9609:	learn: 0.0194518	test: 0.0921247	best: 0.0921247 (9609)	total: 4m 52s	remaining: 11.9s
9610:	learn: 0.0194504	test: 0.0921245	best: 0.0921245 (9610)	total: 4m 52s	remaining: 11.8s
9611:	learn: 0.0194489	test: 0.0921242	best: 0.0921242 (9611)	total: 4m 52s	remaining: 11.8s
9612:	learn: 0.0194457	test: 0.0921240	best: 0.0921240 (9612)	total: 4m 52s	re

9692:	learn: 0.0192378	test: 0.0920656	best: 0.0920651 (9687)	total: 4m 55s	remaining: 9.36s
9693:	learn: 0.0192373	test: 0.0920656	best: 0.0920651 (9687)	total: 4m 55s	remaining: 9.33s
9694:	learn: 0.0192334	test: 0.0920632	best: 0.0920632 (9694)	total: 4m 55s	remaining: 9.3s
9695:	learn: 0.0192303	test: 0.0920638	best: 0.0920632 (9694)	total: 4m 55s	remaining: 9.27s
9696:	learn: 0.0192263	test: 0.0920646	best: 0.0920632 (9694)	total: 4m 55s	remaining: 9.24s
9697:	learn: 0.0192251	test: 0.0920645	best: 0.0920632 (9694)	total: 4m 55s	remaining: 9.21s
9698:	learn: 0.0192195	test: 0.0920617	best: 0.0920617 (9698)	total: 4m 55s	remaining: 9.18s
9699:	learn: 0.0192175	test: 0.0920598	best: 0.0920598 (9699)	total: 4m 55s	remaining: 9.14s
9700:	learn: 0.0192156	test: 0.0920578	best: 0.0920578 (9700)	total: 4m 55s	remaining: 9.11s
9701:	learn: 0.0192150	test: 0.0920572	best: 0.0920572 (9701)	total: 4m 55s	remaining: 9.08s
9702:	learn: 0.0192120	test: 0.0920569	best: 0.0920569 (9702)	total: 4m

9781:	learn: 0.0189998	test: 0.0919988	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.65s
9782:	learn: 0.0189971	test: 0.0919994	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.62s
9783:	learn: 0.0189943	test: 0.0919981	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.59s
9784:	learn: 0.0189920	test: 0.0919973	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.56s
9785:	learn: 0.0189911	test: 0.0919973	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.53s
9786:	learn: 0.0189901	test: 0.0919983	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.5s
9787:	learn: 0.0189890	test: 0.0919985	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.46s
9788:	learn: 0.0189873	test: 0.0919975	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.43s
9789:	learn: 0.0189861	test: 0.0919978	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.4s
9790:	learn: 0.0189839	test: 0.0919983	best: 0.0919970 (9778)	total: 4m 58s	remaining: 6.37s
9791:	learn: 0.0189821	test: 0.0919984	best: 0.0919970 (9778)	total: 4m 

9873:	learn: 0.0187714	test: 0.0919475	best: 0.0919433 (9869)	total: 5m 1s	remaining: 3.84s
9874:	learn: 0.0187693	test: 0.0919444	best: 0.0919433 (9869)	total: 5m 1s	remaining: 3.81s
9875:	learn: 0.0187649	test: 0.0919431	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.78s
9876:	learn: 0.0187633	test: 0.0919436	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.75s
9877:	learn: 0.0187591	test: 0.0919444	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.72s
9878:	learn: 0.0187558	test: 0.0919447	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.69s
9879:	learn: 0.0187514	test: 0.0919436	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.66s
9880:	learn: 0.0187495	test: 0.0919445	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.63s
9881:	learn: 0.0187464	test: 0.0919450	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.6s
9882:	learn: 0.0187435	test: 0.0919440	best: 0.0919431 (9875)	total: 5m 1s	remaining: 3.57s
9883:	learn: 0.0187392	test: 0.0919452	best: 0.0919431 (9875)	total: 5m 1s	remain

9969:	learn: 0.0185325	test: 0.0919067	best: 0.0919056 (9960)	total: 5m 5s	remaining: 918ms
9970:	learn: 0.0185300	test: 0.0919058	best: 0.0919056 (9960)	total: 5m 5s	remaining: 887ms
9971:	learn: 0.0185259	test: 0.0919040	best: 0.0919040 (9971)	total: 5m 5s	remaining: 857ms
9972:	learn: 0.0185258	test: 0.0919042	best: 0.0919040 (9971)	total: 5m 5s	remaining: 826ms
9973:	learn: 0.0185237	test: 0.0919052	best: 0.0919040 (9971)	total: 5m 5s	remaining: 796ms
9974:	learn: 0.0185201	test: 0.0919042	best: 0.0919040 (9971)	total: 5m 5s	remaining: 765ms
9975:	learn: 0.0185169	test: 0.0919017	best: 0.0919017 (9975)	total: 5m 5s	remaining: 734ms
9976:	learn: 0.0185148	test: 0.0919015	best: 0.0919015 (9976)	total: 5m 5s	remaining: 704ms
9977:	learn: 0.0185134	test: 0.0919021	best: 0.0919015 (9976)	total: 5m 5s	remaining: 673ms
9978:	learn: 0.0185130	test: 0.0919018	best: 0.0919015 (9976)	total: 5m 5s	remaining: 643ms
9979:	learn: 0.0185074	test: 0.0919022	best: 0.0919015 (9976)	total: 5m 5s	remai

In [101]:
print(cat_model.learning_rate_)
print(pd.DataFrame(sorted(list(zip(X.columns, cat_model.feature_importances_)), key=lambda x: x[1], reverse=True)))
print("학습(score) :", cat_model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", cat_model.score(X_test, y_test) ) # 결정계수

0.011366999708116055
               0          1
0        단지내주차면수  38.191002
1      단지별전용면적평균   6.128702
2           총세대수   6.106466
3   주차면수 대비 총세대수   6.053489
4            거주율   5.589806
5           단지분류   5.238088
6           공급유형   4.966755
7             지역   4.870449
8           자격유형   4.790653
9           단지코드   3.891987
10     단지코드_Type   3.469567
11           공가수   2.193488
12    10분내버스정류장수   2.185063
13       단지분류lbl   1.244943
14         임대보증금   1.211897
15           임대료   1.065075
16          전용면적   0.912114
17      10분내지하철수   0.839390
18          소득분위   0.600747
19      전용면적별세대수   0.450320
20        임대건물구분   0.000000
학습(score) : 0.9983648363514975
테스트(score) : 0.9860004768328317


## grid_search로 최적의 파라미터 찾기

In [102]:
# grid_search_model = CatBoostRegressor(
#     cat_features=['단지코드', '임대건물구분', '지역', '공급유형', '자격유형'],
#     nan_mode="Forbidden"
# )
# grid = {'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}
# grid_search_result = grid_search_model.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                       #  plot=True,
#                                        )

# print(grid_search_result["params"])
print(max(grid_search_result["cv_results"]["iterations"]))

In [103]:
# print(grid_search_model.learning_rate_)
# print(sorted(list(zip(X.columns, grid_search_model.feature_importances_)), key=lambda x: x[1], reverse=True))
# print("학습(score) :", grid_search_model.score(X_train, y_train) ) # 결정계수
# print("테스트(score) :", grid_search_model.score(X_test, y_test) ) # 결정계수

## 교차검증

In [104]:
# import time
# # now_time=time.time()

# cat_model_cv = CatBoostRegressor(
#     cat_features=['단지코드', '임대건물구분', '지역', '공급유형', '자격유형'],
#     loss_function='MAE',
#     n_estimators=500, 
#     learning_rate=0.05, 
#     random_state=42,
#     task_type="GPU"
# )
# score = cross_val_score(cat_model_cv, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")
# m_score = np.abs(score.mean())  # 절대값
# # pro_time = time.time() - now_time
# print("Catboost Score : {}".format(m_score))  # 점수
# # print("소요시간:",pro_time)  # 걸린 시간

## 제출파일 만들기

In [105]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '단지코드_Type', '소득분위',
       '주차면수 대비 총세대수', '거주율', '단지분류', '단지분류lbl', '전용면적별세대수', '단지별전용면적평균',
       '등록차량수', 'log_등록차량수'],
      dtype='object')

In [106]:
train.dtypes

단지코드             object
총세대수              int64
임대건물구분           object
지역               object
공급유형             object
전용면적            float64
공가수               int64
자격유형             object
임대보증금           float64
임대료             float64
10분내지하철수        float64
10분내버스정류장수        int64
단지내주차면수           int64
단지코드_Type         int64
소득분위              int64
주차면수 대비 총세대수    float64
거주율             float64
단지분류             object
단지분류lbl           int64
전용면적별세대수          int32
단지별전용면적평균       float64
등록차량수           float64
log_등록차량수       float64
dtype: object

In [107]:
X_test = test[train.columns[:-2]]
pred = cat_model.predict(X_test)

In [108]:
test["등록차량수"] = np.expm1(pred)
test

<ipython-input-108-244d79722daa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["등록차량수"] = np.expm1(pred)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,단지내주차면수,단지코드_Type,소득분위,주차면수 대비 총세대수,거주율,단지분류,단지분류lbl,전용면적별세대수,단지별전용면적평균,등록차량수
1605,C1072,754,아파트,경기도,국민임대,39.79,14,H,22830000.0,189840.0,...,683,2,2,0.905836,0.981432,중단지,2,116,48.269164,757.511882
1606,C1072,754,아파트,경기도,국민임대,46.81,14,A,36048000.0,249930.0,...,683,2,3,0.905836,0.981432,중단지,2,30,48.269164,781.115443
1607,C1072,754,아파트,경기도,국민임대,46.90,14,H,36048000.0,249930.0,...,683,2,2,0.905836,0.981432,중단지,2,232,48.269164,782.435073
1608,C1072,754,아파트,경기도,국민임대,51.46,14,H,43497000.0,296780.0,...,683,2,2,0.905836,0.981432,중단지,2,60,48.269164,780.051657
1609,C1072,754,아파트,경기도,국민임대,51.71,14,H,43497000.0,296780.0,...,683,2,2,0.905836,0.981432,중단지,2,51,48.269164,787.204279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,C1267,675,아파트,경상남도,행복주택,26.85,38,L,0.0,0.0,...,467,0,4,0.691852,0.943704,중단지,2,74,32.054015,406.228284
2193,C1267,675,아파트,경상남도,행복주택,36.77,38,L,0.0,0.0,...,467,0,4,0.691852,0.943704,중단지,2,126,32.054015,409.937915
2194,C2189,382,아파트,전라북도,국민임대,29.19,45,H,6872000.0,106400.0,...,300,0,2,0.785340,0.882199,소단지,1,116,36.986230,131.192516
2195,C2189,382,아파트,전라북도,국민임대,39.45,45,H,13410000.0,144600.0,...,300,0,2,0.785340,0.882199,소단지,1,202,36.986230,125.243076


In [109]:
test["등록차량수*전용면적별세대수"] = test['등록차량수'] * test['전용면적별세대수'] / test["총세대수"]
test["단지별차량수평균"] = test.groupby("단지코드")["등록차량수*전용면적별세대수"].transform(np.sum)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index(drop=True)
test_new.head()

<ipython-input-109-ef1a44636ca1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["등록차량수*전용면적별세대수"] = test['등록차량수'] * test['전용면적별세대수'] / test["총세대수"]
<ipython-input-109-ef1a44636ca1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["단지별차량수평균"] = test.groupby("단지코드")["등록차량수*전용면적별세대수"].transform(np.sum)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,소득분위,주차면수 대비 총세대수,거주율,단지분류,단지분류lbl,전용면적별세대수,단지별전용면적평균,등록차량수,등록차량수*전용면적별세대수,단지별차량수평균
0,C1072,754,아파트,경기도,국민임대,39.79,14,H,22830000.0,189840.0,...,2,0.905836,0.981432,중단지,2,116,48.269164,757.511882,116.540290,780.191992
1,C1128,1354,아파트,경기도,국민임대,39.79,9,H,22830000.0,189840.0,...,2,0.898080,0.993353,대단지,3,398,46.846145,1341.930186,394.452152,1367.777848
2,C1456,619,아파트,부산광역시,국민임대,33.40,18,A,19706000.0,156200.0,...,3,0.883683,0.970921,중단지,2,82,38.651696,521.320633,69.060245,471.560763
3,C1840,593,아파트,전라북도,국민임대,39.57,7,A,14418000.0,108130.0,...,3,0.915683,0.988196,소단지,1,335,43.471619,510.863113,288.598892,510.854521
4,C1332,1297,아파트,경기도,국민임대,39.99,11,H,28598000.0,203050.0,...,2,0.857363,0.991519,대단지,3,425,45.841735,1423.865063,466.571050,1418.883834


In [110]:
test_new.shape

(147, 24)

In [111]:
add_dat={'code':['C2675','C2335','C1327'],'num':['0','0','0']}
add_df=pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [112]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,780.192
1,C1128,1367.78
2,C1456,471.561
3,C1840,510.855
4,C1332,1418.88
...,...,...
145,C1267,378.539
146,C2189,125.24
147,C2675,0
148,C2335,0


In [113]:
sub_df.to_csv('./'+input("파일 이름을 지정해주세요(.csv 제외하고 입력):") + '.csv', index=False)
sub_df.head()

파일 이름을 지정해주세요(.csv 제외하고 입력):0729_new_baseline_3rd_피처추가


,code,num
0,C1072,780.192
1,C1128,1367.78
2,C1456,471.561
3,C1840,510.855
4,C1332,1418.88
